This notebook aims to recreate an annealer machine running simulated quantum annealing.

See https://doi.org/10.1109/ICRC.2017.8123652

In [1]:
import numpy as np
import scipy as sp
import numba as nb
import time
from scipy.sparse import bsr_matrix

In [14]:
def one_SQA_run(J, trans_fld_sched, M, T, ansatz_state=None):
    """
    One simulated quantum annealing run over the full transverse field strength schedule.
    The goal is to find a state such that sum(J[i, i]*state[i]) + sum(J[i, j]*state[i]*state[j]) is minimized.
    
    Parameters:
        J (2-D array of float): The matrix representing the local and coupling field of the problem.
                                Local fields should be on the diagonal of the input matrix.
        trans_fld_sched (list[float]): The transeverse field strength schedule for QA.
                                       The number of iterations is implicitly the length of temp_schedule.
        M (int): The number of Trotter replicas. Larger M leads to higher probability of finding ground state.
        T (float): Temperature parameter. Smaller T leads to higher probability of finding ground state.
        ansatz_state (1-D array of bool, default=None): The boolean vector representing the initial state.
                                                        If None, a random state is chosen.
    
    Return: final_state (1-D array of bool)
    """
    
    N = J.shape[0]
    J = 0.5*(J + J.T) # making sure J is symmetric
    J = np.kron(np.eye(M), J/M) # block diagonal of J, repeated M times
    Jp_terms = np.eye(N*M, k=N) + np.eye(N*M, k=N*(1-M))
    Jp_terms = 0.5*(Jp_terms + Jp_terms.T)
    
    Q = 4*J - 6*np.diag(np.diag(J)) + 4*np.diag(np.sum(J, axis=0))
    Qp_terms = 4*Jp_terms + 4*np.eye(N*M)
    
    if ansatz_state is None:
        state = (np.random.binomial(1, 0.5, N*M) == 1)
    else:
        state = np.tile(ansatz_state, M)
    
    
    for Gamma in trans_fld_sched:
        Jp_coef = -0.5 * T * np.log(np.tanh(Gamma / M / T))
        
        # Local move
        flip = np.random.randint(N*M)
        delta_E = 2 * (1 - 2*state[flip]) * np.sum((Q + Jp_coef * Qp_terms)[flip][state]) + Q[flip, flip]
        if np.random.binomial(1, np.minimum(np.exp(-delta_E/T), 1.)):
            state[flip] ^= True
        
        # Global move
        flip = np.random.randint(N)
        delta_E = 0
        for i in range(M):
            delta_E += 2 * (1 - 2*state[flip + i*N]) * np.sum(Q[flip + i*N][state]) + Q[flip + i*N, flip + i*N]
        if np.random.binomial(1, np.minimum(np.exp(-delta_E/T), 1.)):
            for i in range(M):
                state[flip + i*N] ^= True
    
    return state

In [9]:
Q = np.reshape(np.array(range(16)), (4,  4))

In [10]:
Q

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [11]:
flip = np.array([0, 2])

In [12]:
state = np.array([0, 1, 0, 1])

In [13]:
state[flip]

array([0, 0])

According to https://doi.org/10.1103/PhysRevB.66.094203, M\*T should be on the order of coupling strengths |J|, but not smaller.

In [15]:
J = np.array([[-1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.], [0., 0., 0., 1.]])
ansatz = np.zeros(J.shape[0], dtype=np.bool_)

steps_per_spin = 10**3
M = 40
T = 0.05

steps = steps_per_spin * M * J.shape[0]
Gamma0 = 3
schedule = np.linspace(Gamma0, 10**(-8), num=steps)

In [16]:
np.random.seed(0)
start_time = time.time()
ans = one_SQA_run(J, schedule, M, T)
total_time = time.time() - start_time
print(f'ground state: {ans}; time: {total_time} s')

ground state: [False False False False  True False  True False False False False False
  True False False False False False False False  True False False False
  True  True False False  True False False  True  True False False False
 False  True  True False False False False  True False False False False
  True False False False False False False False False False False False
 False False  True False  True False False False  True False False False
  True False False  True False False False False  True False  True  True
  True False False False  True False False False False False False False
 False False False  True False False False False  True False False False
 False False False False False  True False False False False False False
  True False False False False False False False  True False  True False
 False False False False  True  True False False  True False False False
  True  True  True  True  True False False False  True False False  True
 False False False False]; time: 261.

In [17]:
for i in range(J.shape[0]):
    print(f"Percentage of +1 for spin {i+1}: {np.sum(ans[i::J.shape[0]])/M:.1%}")

Percentage of +1 for spin 1: 52.5%
Percentage of +1 for spin 2: 12.5%
Percentage of +1 for spin 3: 15.0%
Percentage of +1 for spin 4: 17.5%


In [22]:
np.sum(np.reshape(ans, (M, J.shape[0])), axis=0) >= 0.5*M

array([ True, False, False, False])